# Entraînement du Modèle Morningstar (Version Locale)

Ce notebook entraîne le modèle `enhanced_hybrid_model` localement en utilisant les données préparées pour un actif spécifique (par défaut 'sol').

In [ ]:
# Configuration de base
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path('/home/morningstar/Desktop/crypto_robot/Morningstar')
os.chdir(PROJECT_ROOT)
print(f"Répertoire projet: {PROJECT_ROOT}")

In [ ]:
# Installation des dépendances (si nécessaire)
# !pip install -r requirements.txt

In [ ]:
# Chargement des données et préparation des labels
import tensorflow as tf
import numpy as np
import pandas as pd
from model.training.data_loader import load_and_split_data
from model.architecture.enhanced_hybrid_model import build_enhanced_hybrid_model

ASSET_NAME = 'sol'
DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / f'{ASSET_NAME}_final.parquet'

# Vérification des colonnes disponibles
print("Colonnes disponibles dans le fichier:")
print(pd.read_parquet(DATA_PATH).columns.tolist())

# Chargement des données brutes
data = pd.read_parquet(DATA_PATH)

# Renommage des colonnes pour correspondre aux noms de sortie du modèle
data = data.rename(columns={
    'trading_signal': 'signal',
    'volatility': 'volatility_quantiles',
    'market_regime': 'market_regime'
})

# Création de volatility_regime si nécessaire
if 'volatility_regime' not in data.columns:
    data['volatility_regime'] = data['market_regime']  # Utiliser market_regime comme placeholder

# Conversion des régimes textuels en entiers
regime_map = {'bearish_placeholder': 0, 'sideways_placeholder': 1, 'bullish_placeholder': 2}
if 'market_regime' in data.columns and data['market_regime'].dtype == 'object':
    data['market_regime'] = data['market_regime'].map(regime_map).fillna(-1).astype(int)
if 'volatility_regime' in data.columns and data['volatility_regime'].dtype == 'object':
    data['volatility_regime'] = data['volatility_regime'].map(regime_map).fillna(-1).astype(int)

# Préparation de sl_tp à partir de level_sl et level_tp
if 'level_sl' in data.columns and 'level_tp' in data.columns:
    data['level_sl'] = pd.to_numeric(data['level_sl'], errors='coerce').fillna(0)
    data['level_tp'] = pd.to_numeric(data['level_tp'], errors='coerce').fillna(0)
else:
    print("Warning: Colonnes level_sl ou level_tp manquantes")

# Sauvegarde temporaire pour charger avec data_loader
temp_path = PROJECT_ROOT / 'data' / 'processed' / 'temp.parquet'
data.to_parquet(temp_path)

# Chargement avec la nouvelle version de data_loader
output_names = ['signal', 'volatility_quantiles', 'volatility_regime', 'market_regime', 'sl_tp']
X, y_dict = load_and_split_data(
    file_path=temp_path,
    label_columns=output_names,
    as_tensor=True
)

# Suppression du fichier temporaire
temp_path.unlink()

In [ ]:
# Séparation train/validation
num_samples = X.shape[0]
num_val = int(num_samples * 0.2)
X_train, X_val = X[:num_samples-num_val], X[num_samples-num_val:]
y_train = {k: v[:num_samples-num_val] for k, v in y_dict.items()}
y_val = {k: v[num_samples-num_val:] for k, v in y_dict.items()}

In [ ]:
# Préparation des inputs pour modèle hybride (avec placeholder pour LLM)
print("\nPréparation des dictionnaires d'input pour le modèle hybride...")
llm_embedding_dim = 768 # Doit correspondre à la définition du modèle

llm_placeholder_train = np.zeros((X_train.shape[0], llm_embedding_dim), dtype=np.float32)
X_train_dict = {
    'technical_input': X_train,
    'llm_input': llm_placeholder_train
}

llm_placeholder_val = np.zeros((X_val.shape[0], llm_embedding_dim), dtype=np.float32)
X_val_dict = {
    'technical_input': X_val,
    'llm_input': llm_placeholder_val
}
print("Dictionnaires d'inputs créés.")

In [ ]:
# Initialisation du modèle
input_shape = (X_train.shape[1],)
# Déterminer dynamiquement le nombre de classes si possible
num_signal_classes = int(tf.reduce_max(y_train['signal'])) + 1 if 'signal' in y_train else 2
num_market_regime_classes = int(tf.reduce_max(y_train['market_regime'])) + 1 if 'market_regime' in y_train else 3

model = build_enhanced_hybrid_model(
    input_shape=input_shape,
    llm_embedding_dim=llm_embedding_dim,
    num_trading_classes=num_signal_classes,
    num_regime_classes=num_market_regime_classes
)

# Configuration de la compilation
losses = {
    'signal': 'sparse_categorical_crossentropy',
    'volatility_quantiles': 'mse',
    'volatility_regime': 'sparse_categorical_crossentropy',
    'market_regime': 'sparse_categorical_crossentropy',
    'sl_tp': 'mse'
}

metrics = {
    'signal': ['accuracy'],
    'volatility_quantiles': ['mae'],
    'volatility_regime': ['accuracy'],
    'market_regime': ['accuracy'],
    'sl_tp': ['mae']
}

model.compile(
    optimizer='adam',
    loss=losses,
    metrics=metrics
)

In [ ]:
# Entraînement du modèle
EPOCHS = 50
BATCH_SIZE = 32
MODEL_PATH = PROJECT_ROOT / 'model' / 'training' / 'morningstar_local.h5'

history = model.fit(
    x=X_train_dict,  # Utiliser le dictionnaire d'inputs
    y=y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_dict, y_val), # Utiliser les dictionnaires pour la validation aussi
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath=MODEL_PATH,
            save_best_only=True,
            monitor='val_loss'
        ),
        tf.keras.callbacks.EarlyStopping(
            patience=10,
            restore_best_weights=True
        )
    ]
)

In [ ]:
# Évaluation et visualisation des résultats
results = model.evaluate(X_val_dict, y_val) # Utiliser le dictionnaire pour l'évaluation
print("Résultats d'évaluation:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")